In [11]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")
Pkg.add("Gruobi")
Pkg.add("DataFrames")
Pkg.add("CSV")

    Updating registry at `C:\Users\janle\.julia\registries\General.toml`


   Resolving package versions...


  No Changes to `C:\Users\janle\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\janle\.julia\environments\v1.9\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\janle\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\janle\.julia\environments\v1.9\Manifest.toml`

Pkg.Types.PkgError: The following package names could not be resolved:
 * Gruobi (not found in project, manifest or registry)
[36m   Suggestions:[39m [0m[1mG[22mene[0m[1mr[22malizedD[0m[1mu[22mbinsIntervalPr[0m[1mo[22m[0m[1mb[22mlem

In [12]:
using JuMP, GLPK
using DataFrames
using CSV

In [13]:
#load demand data
demand = CSV.File("../data/demand.csv") |> DataFrame

#load cost data
cost = CSV.File("../data/cost_generation.csv") |> DataFrame

#load capacity data
capacity = CSV.File("../data/technical_data_generation.csv") |> DataFrame

#load wind_technicaldata
wind = CSV.File("../data/wind_technicaldata.csv") |> DataFrame


Row,Unit,Node,Pmax (MW),Pmin (MW),Ci ($/MWh)
,Int64,Int64,Int64,Int64,Int64
1,1,3,200,0,0
2,2,5,200,0,0
3,3,7,200,0,0
4,4,16,200,0,0
5,5,21,200,0,0
6,6,23,200,0,0


In [14]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], objective_value = Float64[])

# For x variables, manually add each column. This is a one-time setup.
for i in 1:(size(cost, 1)+size(wind, 1))
    result_df[!, Symbol("x$i")] = Float64[]
end

In [19]:

for hour in 1:24
  # Create a new model with GLPK solver
  model = Model(GLPK.Optimizer)

  # Define the decision variables for every generator
  unregister(model, :x)  # Unregister the existing variable named "x" from the model
  
  #demand bids
  name = "demand_hour_" * string(hour-1) * ".csv"
  demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame
   
  # number of convential generators
   G = size(cost, 1)

   # number of demand
   D = size(demand_bids, 1)
 
   # number of wind generators
   W = size(wind, 1)


  # decision variables for convential generators
  @variable(model, x[1:G])  # Create the new variable
  for generator in 1:G
      set_lower_bound(x[generator], capacity[generator,4])  # Set the lower bound
      set_upper_bound(x[generator], capacity[generator,3])  # Set the upper bound
  end

  # wind decision variables
  @variable(model, x_wind[1:W])  # Create the new variable
  for generator in 1:W
      set_lower_bound(x_wind[generator], wind[generator,4])  # Set the lower bound
      set_upper_bound(x_wind[generator], wind[generator,3])  # Set the upper bound
  end

  # decision variables for demand
  @variable(model, y[1:size(demand_bids, 1)])  # Create the new variable
  for bid in 1:D
      set_lower_bound(y[bid], 0)  # Set the lower bound
      set_upper_bound(y[bid], demand_bids[bid,2])  # Set the upper bound
  end


  # Define the objective function
  @objective(model, Max, sum(demand_bids[d,3] * y[d] for d in D) - sum(cost[g,2] * x[g] for g in 1:G) - sum(wind[w,2] * x_wind[w] for w in 1:W))

  # Add the constraint for the balance between supply and demand
  @constraint(model, sum(x[g] for g in 1:G) + sum(x_wind[w] for w in 1:W) - sum(y[d] for d in D) == 0)

  # Print the model
  #print(model)

  # Solve the model
  optimize!(model)

  # Check the status of the solution
  status = termination_status(model)
  if status == MOI.OPTIMAL
      println("Hour ", hour, ": Optimal solution found")

      # Assuming 'model' and 'x' are properly defined and 'hour' is known
      new_row = [hour, objective_value(model)]
      append!(new_row, [value(x[i]) for i in 1:size(cost, 1)]) # Simplifies adding x values

      push!(result_df, new_row)

  elseif status == MOI.INFEASIBLE
      println("Hour ", hour, ": No feasible solution found")
  else
      println("Hour ", hour, ": Solver status: ", status)
  end
end

Hour 1: No feasible solution found
Hour 2: No feasible solution found
Hour 3: No feasible solution found
Hour 4: No feasible solution found
Hour 5: No feasible solution found
Hour 6: No feasible solution found
Hour 7: No feasible solution found
Hour 8: No feasible solution found
Hour 9: No feasible solution found
Hour 10: No feasible solution found
Hour 11: No feasible solution found
Hour 12: No feasible solution found
Hour 13: No feasible solution found
Hour 14: No feasible solution found
Hour 15: No feasible solution found
Hour 16: No feasible solution found
Hour 17: No feasible solution found
Hour 18: No feasible solution found
Hour 19: No feasible solution found
Hour 20: No feasible solution found
Hour 21: No feasible solution found
Hour 22: No feasible solution found
Hour 23: No feasible solution found
Hour 24: No feasible solution found


In [16]:

    @variable(model, x[1:size(cost, 1)])  # Create the new variable
    for generator in 1:size(cost, 1)
        set_lower_bound(x[generator], capacity[generator,4])  # Set the lower bound
        set_upper_bound(x[generator], capacity[generator,3])  # Set the upper bound
    end


    # Define the objective function
    @objective(model, Min, sum(cost[generator,2] * x[generator] for generator in 1:size(cost,1)))

    # Add the equilibrium constraint
    @constraint(model, sum(x[generator] for generator in 1:size(cost,1)) == demand[hour,2])

UndefVarError: UndefVarError: `model` not defined

In [17]:
result_df

Row,hour,objective_value,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [18]:
#save result_df to csv
CSV.write("../results/market_clearing_Jan.csv", result_df)

"../results/market_clearing_Jan.csv"